In [1]:
# good luck!
import pathlib

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import compose, impute, linear_model, metrics, pipeline, preprocessing

In [2]:
INPUT_DIR = pathlib.Path("/kaggle/input/kaust-academy-ai-week-november-2022")
WORKING_DIR = pathlib.Path("/kaggle/working")

# Load data

In [3]:
train_df = pd.read_csv(
    INPUT_DIR / "train.csv",
    index_col="PassengerId",

)
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


#  Divide the training features from the training target¶


In [4]:
train_features = train_df.drop("Transported", axis=1)
train_target = train_df.loc[:, "Transported"]

In [5]:
train_features.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines


# preproccesing


In [6]:
impute.SimpleImputer
compose.make_column_transformer
pipeline.make_pipeline


<function sklearn.pipeline.make_pipeline(*steps, memory=None, verbose=False)>

# feature engineering

In [7]:
#train_features["Journey"] = train_features["HomePlanet"] + train_features["Destination"]
#train_features['Journey'].unique()

In [8]:
train_features['Side'] = train_features['Cabin'].str.strip().str[-1]
train_features['Deck'] = train_features['Cabin'].str.strip().str[0]
train_features['Num'] = train_features['Cabin'].str.strip().str[2]

train_features.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Side,Deck,Num
PassengerId,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,P,B,0
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,S,F,0
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,S,A,0
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,S,A,0
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,S,F,1


In [9]:
df2 = pd.DataFrame().assign(FoodCourt=train_features['FoodCourt'], RoomService=train_features['RoomService'], 
                            ShoppingMall=train_features['ShoppingMall'], 
                            Spa=train_features['Spa'], VRDeck=train_features['VRDeck'], )
train_features['Expenses'] = df2.sum(axis=1, numeric_only= True)
train_features.head()


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Side,Deck,Num,Expenses
PassengerId,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,P,B,0,0.0
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,S,F,0,736.0
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,S,A,0,10383.0
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,S,A,0,5176.0
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,S,F,1,1091.0


In [10]:
print(train_features['Age'].unique())
train_features['Age'] = pd.cut(train_features.Age,bins=[0,2,17,65,99],labels=["Baby","Child","Adult","Elderly",])
train_features.head(25)

[39. 24. 58. 33. 16. 44. 26. 28. 35. 14. 34. 45. 32. 48. 31. 27.  0.  1.
 49. 29. 10.  7. 21. 62. 15. 43. 47.  2. 20. 23. 30. 17. 55.  4. 19. 56.
 nan 25. 38. 36. 22. 18. 42. 37. 13.  8. 40.  3. 54.  9.  6. 64. 67. 61.
 50. 41. 57. 11. 52. 51. 46. 60. 63. 59.  5. 79. 68. 74. 12. 53. 65. 71.
 75. 70. 76. 78. 73. 66. 69. 72. 77.]


,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Side,Deck,Num,Expenses
PassengerId,,,,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,Adult,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,P,B,0,0.0
0002_01,Earth,False,F/0/S,TRAPPIST-1e,Adult,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,S,F,0,736.0
0003_01,Europa,False,A/0/S,TRAPPIST-1e,Adult,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,S,A,0,10383.0
0003_02,Europa,False,A/0/S,TRAPPIST-1e,Adult,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,S,A,0,5176.0
0004_01,Earth,False,F/1/S,TRAPPIST-1e,Child,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,S,F,1,1091.0
0005_01,Earth,False,F/0/P,PSO J318.5-22,Adult,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,P,F,0,774.0
0006_01,Earth,False,F/2/S,TRAPPIST-1e,Adult,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,S,F,2,1584.0
0006_02,Earth,True,G/0/S,TRAPPIST-1e,Adult,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,S,G,0,0.0
0007_01,Earth,False,F/3/S,TRAPPIST-1e,Adult,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,S,F,3,1018.0


In [11]:
boolean_preprocessing = pipeline.make_pipeline(
    impute.SimpleImputer(strategy="most_frequent"),
)

categorical_preprocessing = pipeline.make_pipeline(
    impute.SimpleImputer(strategy="most_frequent"),
    preprocessing.OneHotEncoder(),
)

numeric_preprocessing = pipeline.make_pipeline(
    impute.KNNImputer()
)

data_preprocessing = compose.make_column_transformer(
    (boolean_preprocessing, ["CryoSleep", "VIP"]),
    (categorical_preprocessing, ['Deck','HomePlanet','Destination','Side','Age']),
    (numeric_preprocessing, compose.make_column_selector(dtype_include=np.float64)),
    remainder = "drop",
)


# Train a model

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifier?
RandomForestClassifier

sklearn.ensemble._forest.RandomForestClassifier

In [13]:

_hyperparameters = {
    
     "max_depth":5,
    "max_features":"log2",
    "learning_rate":0.2

}
estimator = GradientBoostingClassifier(**_hyperparameters)

ml_pipeline = pipeline.make_pipeline(
    data_preprocessing,
    estimator
)
_ = ml_pipeline.fit(train_features, train_target)
_train_predictions = ml_pipeline.predict(train_features)
_report = metrics.classification_report(
    _train_predictions,
    train_target
)

print(_report)


              precision    recall  f1-score   support

       False       0.81      0.87      0.84      4027
        True       0.88      0.83      0.86      4666

    accuracy                           0.85      8693
   macro avg       0.85      0.85      0.85      8693
weighted avg       0.85      0.85      0.85      8693



# load test

In [14]:
test_features = pd.read_csv(
    INPUT_DIR / "test.csv",
    index_col="PassengerId",
)
test_features["Journey"] = test_features["HomePlanet"] + test_features["Destination"]

df2 = pd.DataFrame().assign(FoodCourt=test_features['FoodCourt'], RoomService=test_features['RoomService'], 
                            ShoppingMall=test_features['ShoppingMall'], 
                            Spa=test_features['Spa'], VRDeck=test_features['VRDeck'], )
test_features['Expenses'] = df2.sum(axis=1, numeric_only= True)
test_features['Side'] = test_features['Cabin'].str.strip().str[-1]
test_features['Deck'] = test_features['Cabin'].str.strip().str[0]
test_features['Num'] = test_features['Cabin'].str.strip().str[2]
test_features['Age'] = pd.cut(test_features.Age,bins=[0,2,17,65,99],labels=["Baby","Child","Adult","Elderly",])


In [15]:
predictions = ml_pipeline.predict(test_features)
